In [188]:
import pandas as pd
import requests as req

In [189]:
df = pd.read_csv('cat-dm.csv')
df.head()

,pessoacnpj,pessoaid,pessoafantasia,pessoarazaosocial
0,NaN,137,FREDERICO FIGUEIREDO ARAUJO,FREDERICO FIGUEIREDO ARAUJO
1,NaN,138,JOSE MANOEL ARAUJO CARVALHO,JOSE MANOEL ARAUJO CARVALHO
2,NaN,139,GILBERTO MANTOVANI,GILBERTO MANTOVANI
3,NaN,140,SIMONE MARIA CARDOSO,SIMONE MARIA CARDOSO
4,NaN,141,EDILIANI PETKOV,EDILIANI PETKOV


In [190]:
df = df.dropna(how='any')
df.head()

,pessoacnpj,pessoaid,pessoafantasia,pessoarazaosocial
137,00.164.320/0001-87,377,SM COMERCIO DE DOCES LTDA,SM COMERCIO DE DOCES LTDA ME
138,00.362.290/0001-13,136,REIS E CUSTODIO LTDA,REIS E CUSTODIO LTDA
139,00.377.455/0001-20,709,Idexx Brasil,IDEXX BRASIL LABORATORIOS LTDA
140,00.383.974/0001-00,64,CLAUDOMIRO APARECIDO DA SILVA,CLAUDOMIRO APARECIDO DA SILVA
141,00.562.068/0001-64,65,IZONEI DE PAULA ME,IZONEI DE PAULA ME


In [191]:
teste = req.get("https://brasilapi.com.br/api/cnpj/v1/00164320000187")

if teste.status_code == 200:
    print(teste.json())
else:
    print(f"Erro {teste.status_code}: {teste.text}")

{'uf': 'SP', 'cep': '14870010', 'qsa': [{'pais': None, 'nome_socio': 'MARIA MONICA BALLINI FACCO PANOSSO', 'codigo_pais': None, 'faixa_etaria': 'Entre 51 a 60 anos', 'cnpj_cpf_do_socio': '***559928**', 'qualificacao_socio': 'Administrador', 'codigo_faixa_etaria': 6, 'data_entrada_sociedade': '1997-01-02', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 5, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}, {'pais': None, 'nome_socio': 'MAJUPAN PARTICIPACOES LTDA', 'codigo_pais': None, 'faixa_etaria': 'Não se aplica', 'cnpj_cpf_do_socio': '25035076000183', 'qualificacao_socio': 'Sócio', 'codigo_faixa_etaria': 0, 'data_entrada_sociedade': '2018-01-23', 'identificador_de_socio': 1, 'cpf_representante_legal': '***559928**', 'nome_representante_legal': 'MARIA MONICA BALLINI FACCO PANOSSO', 'codigo_qualificacao_socio': 22, 'qualificacao_representante_legal': 'Ad

In [192]:
import time
import pandas as pd
import requests as req

# Limpar o CNPJ no DataFrame antes de iterar
df['pessoacnpj'] = df['pessoacnpj'].astype(str).str.replace(r'\D', '', regex=True)

# Variáveis de controle
MAX_REQUESTS = len(df)
cnaes = []

for i, cnpj in enumerate(df['pessoacnpj']):
    # Garantir que não exceda o limite de solicitações
    if i >= MAX_REQUESTS:
        break

    try:
        # Fazer a solicitação à API
        response = req.get(f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}")
        if response.status_code == 200:
            data = response.json()
            # Adicionar os dados ao resultado
            cnaes.append({
                "Cod": data.get('cnae_fiscal'),
                "Nome": data.get('cnae_fiscal_descricao'),
                "CNPJ": cnpj
            })
        else:
            print(f"Erro {response.status_code}: {response.text}")
    except Exception as e:
        print(f"Erro ao processar o CNPJ {cnpj}: {e}")

    # Pausar entre solicitações
    print(f"{i+1}/{MAX_REQUESTS} requisições executadas...")
    time.sleep(1)

# Converter a lista de CNAEs para DataFrame
cnaes_df = pd.DataFrame(cnaes)

# Visualizar os primeiros resultados
print(cnaes_df.head())

1/676 requisições executadas...
2/676 requisições executadas...
3/676 requisições executadas...
4/676 requisições executadas...
5/676 requisições executadas...
6/676 requisições executadas...
7/676 requisições executadas...
8/676 requisições executadas...
9/676 requisições executadas...
10/676 requisições executadas...
11/676 requisições executadas...
12/676 requisições executadas...
13/676 requisições executadas...
14/676 requisições executadas...
15/676 requisições executadas...
16/676 requisições executadas...
17/676 requisições executadas...
18/676 requisições executadas...
19/676 requisições executadas...
20/676 requisições executadas...
21/676 requisições executadas...
22/676 requisições executadas...
23/676 requisições executadas...
24/676 requisições executadas...
25/676 requisições executadas...
26/676 requisições executadas...
27/676 requisições executadas...
28/676 requisições executadas...
29/676 requisições executadas...
30/676 requisições executadas...
31/676 requisições 

In [193]:
cnaes_df.to_csv('cnaes-dm.csv', index=False, header=True, sep=";")